In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, Sequential, optimizers, losses

In [18]:
# 数据的前处理模块
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [19]:
# 数据的预处理
batchz = 64

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
print(x_train.shape,y_train.shape)
y_train = tf.squeeze(y_train, axis=1)
y_test = tf.squeeze(y_test, axis=1)
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_db = train_db.map(preprocess).shuffle(1000).batch(batchz)
test_db = test_db.map(preprocess).batch(batchz)

example = next(iter(train_db))
print("一个batch维度：", example[0].shape, example[1].shape)
print("x最小值：",tf.reduce_min(example[0]), "x的最大值", tf.reduce_max(example[0]))

(50000, 32, 32, 3) (50000, 1)
一个batch维度： (64, 32, 32, 3) (64,)
x最小值： tf.Tensor(0.0, shape=(), dtype=float32) x的最大值 tf.Tensor(1.0, shape=(), dtype=float32)


In [ ]:
conv_layers = [
    # Conv-Conv-Pooling 单元 1
    # 64 个卷积核，输入输出大小同
    layers.Conv2D(64, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    # 单元2
    layers.Conv2D(128, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    # 单元3
    layers.Conv2D(256, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    # 单元4
    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    # 单元5
    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),

    # 3个全连接层
    layers.Flatten(),
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(10)
]

VGG13_net = Sequential(conv_layers)
VGG13_net.build(input_shape=[None, 32, 32, 3])
VGG13_net.summary()

In [ ]:
optimizer = optimizers.Adam(learning_rate=0.001)
for epoch in range(10):
    for step, (x,y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            # [b, 32, 32, 3]=> [b, 10]
            logits = VGG13_net(x)
            y_onehot = tf.one_hot(y, depth=10)
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss, VGG13_net.variables)
        optimizer.apply_gradients(zip(grads, VGG13_net.variables))
        
        if step % 200 == 0:
            print(epoch, step, '损失：', float(loss))
    
    total_num, total_correct = 0, 0
    
    for x,y in test_db:
        logits = VGG13_net(x,training=False)
        prob = tf.nn.softmax(logits, axis=1)
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)

        correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)

        total_num += x.shape[0]
        total_correct += int(correct)

    acc = total_correct / total_num
    print(epoch, 'acc:', acc) 

此处模型的参数为 900万

训练速度慢，更改 batch_size 变小,64-512